# Project: Customer classification (Online Retail II)

- Date: August 5 2025 -

- Data: This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.

- Description: In this project our goal will be to use this data to detect some similarities between the customers,divide them in some groups with some particular habits or behaviors. This project could be very interresting for marketing strategies. 

In [ ]:
import pandas as pd

# Download the excel file
df = pd.read_excel('/home/christian/ProjetsPerso/IA/MachineLearning/Customer_Classification/data/online_retail_II.xlsx',
                    sheet_name="Year 2010-2011",
                   dtype={"Invoice": str, "StockCode": str, "Customer ID": str},
                   parse_dates=["InvoiceDate"]
                   )

# Information on the data frame
print("Number of customers: ", df["Customer ID"].nunique())
print("Name of columns: ", df.columns)
print("Number of features(colomns):", len(df.columns))
print("shape of the data frame: ", df.shape)


Number of customers:  4372
Name of columns:  Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')
Number of features(colomns): 8
shape of the data frame:  (541910, 8)


# Cleaning the data
Now we will try to clean the data, deleting lines without customers or resigned transactions and to remove duplicate values. We will also add a colimn Totalprice as the multiplication of the quantity by the unity prices.

In [ ]:
# delete line without Customer id
df = df.dropna(subset=["Customer ID"])
print("After drop nan Customer ID :", df.shape)

# delete not finish transaction and negative quantity
mask_return = df["Quantity"] < 0
mask_cancel = df["Invoice"].str.startswith("C")
df = df[~(mask_return | mask_cancel)]
print("After deleting unfinished transaction/neg. quantity :" , df.shape)

# delete duplicate line
df = df.drop_duplicates()
print("After drop_duplicates :", df.shape)

# columns "TotalPrice"
df["TotalPrice"]=df["Quantity"] * df["Price"]
print("After adding 'TotalPrice': ",df.shape)

After drop nan Customer ID : (401605, 8)
After deleting unfinished transaction/neg. quantity : (392733, 8)
After drop_duplicates : (392733, 8)
after adding 'TotalPrice':  (392733, 9)


In [ ]:
["1","2324"]

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

churn risk, 